In [1]:
from google import genai
from google.genai import types
import pathlib
import httpx
import os
import json

In [2]:
# 환경 변수 불러오기
api_key = os.environ.get('MY_API_KEY')
# client 설정
client = genai.Client(api_key=api_key)

In [3]:
# 미리 정의된 스키마, 시스템 프롬포트 불러오기 (파일 이름으로 자동 탐색)
def find_file(filename, search_path='.'):
    # 주어진 파일명을 search_path부터 재귀적으로 탐색하여 경로 반환
    for root, dirs, files in os.walk(search_path):
        if filename in files:
            return os.path.join(root, filename)
    return None

# 상위 디렉토리부터 탐색 시작
# 현재 노트북의 상위 디렉토리부터 탐색 시작
# base_path = pathlib.Path.cwd().parent.parent
# 현재 디렉토리부터 탐색 시작
base_path = pathlib.Path.cwd()

concept_planner_schema_path = find_file("ConceptPlanner_Object.json", base_path)
concept_planner_system_prompt_path = find_file("ConceptPlanner_SystemPrompt.md", base_path)

if not concept_planner_schema_path or not concept_planner_system_prompt_path:
    raise FileNotFoundError("Concept Planner 프롬포트 파일을 찾을 수 없습니다")

with open(concept_planner_schema_path, 'r', encoding='utf-8') as f:
    concept_planner_schema = json.load(f)

with open(concept_planner_system_prompt_path, 'r', encoding='utf-8') as f:
    concept_planner_system_prompt = f.read()

In [7]:
# 입력: 강의 자료, 유저 Profile, target_cont
# Contents 생성하기

lecture_material = "/Users/jhkim/Desktop/LectureTestGenerator/SamSung(260109).pdf"
user_profile = {
    "learning_goal": {
        "focus_areas": [
            "삼성전자 실적",
            "삼성전자 전망"
        ],
        "target_depth": "Deep Understanding",
        "question_modality": "Conceptual"
    },
    "user_status": {
        "proficiency_level": "Intermediate",
        "weakness_focus": True
    },
    "interaction_style": {
        "language_preference": "Korean_with_Korean_Terms",
        "scenario_based": True
    },
    "feedback_preference": {
        "strictness": "Strict",
        "explanation_depth": "Detailed_with_Examples"
    },
    "scope_boundary": "Lecture_Material_Only"
}
# 목표 카드 개수
target_cont = 10

# lecture_material 파일 읽기
path = pathlib.Path(lecture_material)

# 파일 타입별 처리
if path.is_file(): # 실제 존재하는 파일인지 먼저 검사
    # 파일 경로 소문자로 전환 (일관성 높이기 위해)
    suffix = path.suffix.lower()
    # PDF인 경우 처리
    if suffix == '.pdf':
        lecture_content = client.files.upload(file=path)
    # 텍스트 파일인 경우 처리
    elif suffix in ['.txt', '.md']:
        lecture_content = path.read_text(encoding='utf-8')
    # 지원되지 않는 파일은 경우
    else:
        lecture_content = "강의 자료 없음"
else:
    # 파일이 없다면, 파일 경로가 아닌, 강의 텍스트가 입력되었다고 처리
    lecture_content = lecture_material


ConceptPlanner_contents = [
    lecture_content,
    "User Profile: " + json.dumps(user_profile),
    "Target Content Count: " + str(target_cont)
]

In [8]:
# Response 생성
response = client.models.generate_content(
    model = "gemini-3-flash-preview",
    contents = ConceptPlanner_contents,
    config={
        "system_instruction": concept_planner_system_prompt,
        "response_mime_type": "application/json",
        "response_schema": concept_planner_schema
    })

In [9]:
print(response.text)

{
  "planning_strategy": "학습자의 'Intermediate' 수준과 'Deep Understanding' 목표를 고려하여, 단순히 실적 수치를 나열하기보다는 각 사업부별 실적 변동의 '인과관계(Causal Relationship)'와 향후 시장 전망에 대한 '시나리오(Application_Scenario)' 분석을 중심으로 10개의 카드를 구성함.",
  "planned_items": [
    {
      "id": 1,
      "target_topic": "2025년 4분기 실적 총평",
      "intent_type": "Definition",
      "complexity_level": "Basic",
      "source_reference_hint": "매출액 93조원(전분기 대비 8.0% 증가) 및 영업이익 20.3조원을 기록했으나, 이전 전망치보다는 감소한 규모임."
    },
    {
      "id": 2,
      "target_topic": "DS(Device Solutions) 부문 영업이익 개선 요인",
      "intent_type": "Causal_Relationship",
      "complexity_level": "Intermediate",
      "source_reference_hint": "DRAM 및 SSD 수요 증가에 따른 가격 상승(ASP 상승) 효과와 HBM 물량 증가가 주요 원인임."
    },
    {
      "id": 3,
      "target_topic": "MX(Mobile eXperience) 부문 수익성 악화 원인",
      "intent_type": "Causal_Relationship",
      "complexity_level": "Intermediate",
      "source_reference_hint": "역설적으로 메모리 가격 상승이 원가 구조를 악화시켜 예상치 대비 실적이 크게 부진함."
    },
    {
     